In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.externals import joblib
import time

/home/veruska/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [4]:
data_path = 'data/output/normalized_data_X.csv'
y_path = 'data/output/y.csv'

data = pd.read_csv(data_path)
y = pd.read_csv(y_path)
print(data.shape)
print(y.shape)
data.head()

(1432633, 420)
(1432633, 1)


,route,tripNum,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,...,alertTypeSB_ACCIDENT,alertTypeSB_CHIT_CHAT,alertTypeSB_HAZARD,alertTypeSB_JAM,alertTypeSB_NORMAL,alertTypeSB_POLICE,alertTypeSB_ROAD_CLOSED,jamBlockTypeSB_-,jamBlockTypeSB_NORMAL,jamBlockTypeSB_ROAD_CLOSED_EVENT
0,0.118068,0.066667,0.085019,0.264600,0.496150,0.057584,0.0,0.000000,0.266155,0.496047,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.118068,0.066667,0.085026,0.270218,0.501802,0.064231,0.0,0.000148,0.271812,0.501642,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.118068,0.066667,0.085033,0.273653,0.505296,0.068301,0.0,0.000000,0.275187,0.505192,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.118068,0.066667,0.085039,0.277433,0.509404,0.072900,0.0,0.000149,0.280622,0.510132,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.118068,0.066667,0.085053,0.280050,0.513166,0.077983,0.0,0.000155,0.281878,0.513178,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
# Making training and test data: 80% Training, 20% Test
random.seed(15) #to get always the same set
train_X, test_X, train_Y, test_Y = train_test_split(data, y, test_size=0.20, random_state=7)
train_X_1, train_X_2, train_Y_1, train_Y_2 = train_test_split(train_X, train_Y, test_size=0.50, random_state=7) 

In [4]:
#1%
first_1perc_X = train_X_2[0:350]
second_1perc_X = train_X_2[351:701]
third_1perc_X = train_X_2[702:1052]
first_1perc_Y = train_Y_2[0:350]
second_1perc_Y = train_Y_2[351:701]
third_1perc_Y = train_Y_2[702:1052]

In [ ]:
#10%
first_1perc_X = train_X_2[0:143263]
second_1perc_X = train_X_2[143264:286527]
third_1perc_X = train_X_2[286528:429791]
first_1perc_Y = train_Y_2[0:143263]
second_1perc_Y = train_Y_2[143264:286527]
third_1perc_Y = train_Y_2[286528:429791]

In [5]:
print(first_1perc_X.shape, first_1perc_Y.shape)
print(second_1perc_X.shape, second_1perc_Y.shape)
print(third_1perc_X.shape, third_1perc_Y.shape)

(350, 420) (350, 1)
(350, 420) (350, 1)
(350, 420) (350, 1)


### Stacking

In [4]:
rf = joblib.load('../data/Saved_RF_100_5_08_BB_class.pkl')
rf_pred = rf.predict(train_X_1) #train_X

train_X_1['rf'] = rf_pred
train_X_2['rf'] = rf_pred

/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.21.3 when u

In [5]:
rf_xgboost_stac = XGBClassifier(
     learning_rate =0.1,
     n_estimators=120, 
     max_depth=50,
     min_child_weight=1,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread=-1,
     scale_pos_weight=1,
     seed=23)

rf_xgboost_stac.fit(train_X_1, train_Y_1)
xgb_pred = rf_xgboost_stac.predict(train_X_1)
train_X_1['xgboost'] = xgb_pred
train_X_2['xgboost'] = xgb_pred

/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska_santos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [8]:
rf_xgboost_cb_v1 = CatBoostClassifier(
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1'],
    iterations=10000,
    learning_rate=0.9,
    random_seed=42,
    depth=8,
    l2_leaf_reg=5,
    logging_level='Silent'
)

rf_xgboost_cb_v1.fit(train_X_1, train_Y_1)
rf_xgboost_cb_v1.save_model('cat_1.model')


#Incremetal Learning
rf_xgboost_cb_final = CatBoostClassifier(
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1'],
    iterations=10000,
    learning_rate=0.9,
    random_seed=42,
    depth=8,
    l2_leaf_reg=5,
    logging_level='Silent'
)

rf_xgboost_cb_final.fit(train_X_2, train_Y_2, init_model='cat_1.model')


rf_pred_test = rf.predict(test_X)
test_X['rf'] = rf_pred_test
xgb_pred_test = rf_xgboost_stac.predict(test_X)
test_X['xgboost'] = xgb_pred_test

pred_array = rf_xgboost_cb_final.predict(test_X)
pred_array

/home/veruska_santos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/veruska_santos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


array([0., 0., 0., ..., 0., 0., 0.])

In [13]:
# Bus Bunching
accuracy = accuracy_score(test_Y.busBunching.values, pred_array)
precision = precision_score(test_Y.busBunching.values, pred_array)
recall = recall_score(test_Y.busBunching.values, pred_array)
f_measure = f1_score(test_Y.busBunching.values, pred_array)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9204682281250982
Precision: 0.7592866292245541
Recall: 0.5992689944785753
F-measure: 0.6698539638386647


In [9]:
# saving models
joblib.dump(rf_xgboost_cb_final, 'Saved_stacking_IL_RF_XG_CB.pkl')

# saving predictions
test_Y['predStack'] = pred_array
test_Y.to_csv("Saved_stacking_IL_RF_XG_CB_testY_pred.csv", index=False)

/home/veruska_santos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [12]:
test_Y.busBunching.values

array([0., 1., 0., ..., 0., 0., 0.])

### Majoriting Voting

In [16]:
def get_quality(pred):
    accuracy = accuracy_score(test_Y.busBunching.values, pred)
    precision = precision_score(test_Y.busBunching.values, pred)
    recall = recall_score(test_Y.busBunching.values, pred)
    f_measure = f1_score(test_Y.busBunching.values, pred)
    
    print("Accuracy: " + str(accuracy))
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("F-measure: " + str(f_measure))

#### CatBoost

In [8]:
model_1 = CatBoostClassifier(
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1'],
    iterations=10000,
    learning_rate=0.9,
    random_seed=42,
    depth=8,
    l2_leaf_reg=5,
    logging_level='Silent'
)
model_1.fit(train_X_1, train_Y_1)

model_1.save_model('cat_1.model')

In [17]:
model_2_v2 = CatBoostClassifier(
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1'],
    iterations=10000,
    learning_rate=0.9,
    random_seed=42,
    depth=8,
    l2_leaf_reg=5,
    logging_level='Silent'
)

cat1_time = time.time()
model_2_v2.fit(first_1perc_X, first_1perc_Y, init_model='cat_1.model')
print("Execution time (min): " + str(time.time() - cat1_time))

model_2_v2.save_model('cat_2.model')

Execution time (min): 96.1477279663086


In [25]:
model_2_v3 = CatBoostClassifier(
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1'],
    iterations=10000,
    learning_rate=0.9,
    random_seed=42,
    depth=8,
    l2_leaf_reg=5,
    logging_level='Silent'
)
cat2_time = time.time()
model_2_v3.fit(second_1perc_X, second_1perc_Y, init_model='cat_2.model')
print("Execution time (min): " + str(time.time() - cat2_time))

model_2_v3.save_model('cat_3.model')

Execution time (min): 96.94785022735596


In [33]:
model_2_v4 = CatBoostClassifier(
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1'],
    iterations=10000,
    learning_rate=0.9,
    random_seed=42,
    depth=8,
    l2_leaf_reg=5,
    logging_level='Silent'
)
cat3_time = time.time()
model_2_v4.fit(third_1perc_X, third_1perc_Y, init_model='cat_3.model')
print("Execution time (min): " + str(time.time() - cat3_time))

Execution time (min): 97.91175985336304


In [34]:
# Bus Bunching 50%-3% all data
cat_pred = model_2_v4.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.8904466245763925
Precision: 0.6668678894822382
Recall: 0.3722684501127615
F-measure: 0.47780809156241677


In [26]:
# Bus Bunching 50%-2% all data
cat_pred = model_2_v3.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.8981317641967423
Precision: 0.7285429141716567
Recall: 0.38792544780568733
F-measure: 0.5062755844243716


In [18]:
# Bus Bunching 50%-1% all data
cat_pred = model_2_v2.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9057122016424282
Precision: 0.7792916847852346
Recall: 0.4180988672006636
F-measure: 0.5442183756790497


In [9]:
# Bus Bunching 50% all data
cat_pred = model_1.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9291096476073808
Precision: 0.7820152549177037
Recall: 0.6564533271120098
F-measure: 0.7137542277339346


In [26]:
cat_pred = model_2_v4.predict(test_X)

# Bus Bunching 50%-3%
accuracy = accuracy_score(test_Y.busBunching.values, cat_pred)
precision = precision_score(test_Y.busBunching.values, cat_pred)
recall = recall_score(test_Y.busBunching.values, cat_pred)
f_measure = f1_score(test_Y.busBunching.values, cat_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9379695213565142
Precision: 0.7379110251450677
Recall: 0.5614422369389257
F-measure: 0.63769327204346


In [18]:
cat_pred = model_2_v3.predict(test_X)

# Bus Bunching 50%-2%
accuracy = accuracy_score(test_Y.busBunching.values, cat_pred)
precision = precision_score(test_Y.busBunching.values, cat_pred)
recall = recall_score(test_Y.busBunching.values, cat_pred)
f_measure = f1_score(test_Y.busBunching.values, cat_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9425484724905201
Precision: 0.7477718360071302
Recall: 0.6173657100809419
F-measure: 0.6763401854091092


In [11]:
cat_pred = model_2_v2.predict(test_X)

# Bus Bunching 50%-1%
accuracy = accuracy_score(test_Y.busBunching.values, cat_pred)
precision = precision_score(test_Y.busBunching.values, cat_pred)
recall = recall_score(test_Y.busBunching.values, cat_pred)
f_measure = f1_score(test_Y.busBunching.values, cat_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9479859769621521
Precision: 0.8257731958762886
Recall: 0.5894039735099338
F-measure: 0.6878488621726063


In [6]:
cat_pred = model_2_v2.predict(test_X)

# Bus Bunching 50%-50%
accuracy = accuracy_score(test_Y.busBunching.values, cat_pred)
precision = precision_score(test_Y.busBunching.values, cat_pred)
recall = recall_score(test_Y.busBunching.values, cat_pred)
f_measure = f1_score(test_Y.busBunching.values, cat_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9291131376798696
Precision: 0.7617805549185966
Recall: 0.6889338206703476
F-measure: 0.7235282107125842


In [34]:
cat_pred = model_1.predict(test_X)

# Bus Bunching 50%
accuracy = accuracy_score(test_Y.busBunching.values, cat_pred)
precision = precision_score(test_Y.busBunching.values, cat_pred)
recall = recall_score(test_Y.busBunching.values, cat_pred)
f_measure = f1_score(test_Y.busBunching.values, cat_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9567146025613508
Precision: 0.851024208566108
Recall: 0.6725533480500367
F-measure: 0.7513357994245787


#### XGBoost

In [10]:
xgb_1 = XGBClassifier(
     learning_rate =0.1,
     n_estimators=120, 
     max_depth=50,
     min_child_weight=1,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread=-1,
     scale_pos_weight=1,
     seed=23)
xgb_1.fit(train_X_1, train_Y_1)

xgb_1.save_model('xgb_1.model')

/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
xgb_2_v2 = XGBClassifier(
     learning_rate =0.1,
     n_estimators=120, 
     max_depth=50,
     min_child_weight=1,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread=-1,
     scale_pos_weight=1,
     seed=23)
xgb1_time = time.time()
xgb_2_v2.fit(first_1perc_X, first_1perc_Y, xgb_model='xgb_1.model')
print("Execution time (min): " + str(time.time() - xgb1_time))
xgb_2_v2.save_model('xgb_2.model')

/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Execution time (min): 4.232481479644775


In [27]:
xgb_2_v3 = XGBClassifier(
     learning_rate =0.1,
     n_estimators=120, 
     max_depth=50,
     min_child_weight=1,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread=-1,
     scale_pos_weight=1,
     seed=23)
xgb2_time = time.time()
xgb_2_v3.fit(second_1perc_X, second_1perc_Y, xgb_model='xgb_2.model')
print("Execution time (min): " + str(time.time() - xgb2_time))
xgb_2_v3.save_model('xgb_3.model')

/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Execution time (min): 3.8110358715057373


In [35]:
xgb_2_v4 = XGBClassifier(
     learning_rate =0.1,
     n_estimators=120, 
     max_depth=50,
     min_child_weight=1,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread=-1,
     scale_pos_weight=1,
     seed=23)
xgb3_time = time.time()
xgb_2_v4.fit(third_1perc_X, third_1perc_Y, xgb_model='xgb_3.model')
print("Execution time (min): " + str(time.time() - xgb3_time))

/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Execution time (min): 3.822972059249878


In [36]:
# Bus Bunching 50%-3% all data
xgb_pred = xgb_2_v4.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.910507561242047
Precision: 0.823423513527029
Recall: 0.42683464240350466
F-measure: 0.562228975313279


In [28]:
# Bus Bunching 50%-2% all data
xgb_pred = xgb_2_v3.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9150830462748711
Precision: 0.8164652567975831
Recall: 0.47637193146175183
F-measure: 0.6016861750020464


In [20]:
# Bus Bunching 50%-1% all data
xgb_pred = xgb_2_v2.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9209568382735309
Precision: 0.8319509857041637
Recall: 0.517432667133266
F-measure: 0.6380373981141122


In [11]:
# Bus Bunching 50% all data
xgb_pred = xgb_1.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9325299186464103
Precision: 0.879271609317725
Recall: 0.5782720273738238
F-measure: 0.6976918746481516


In [28]:
xgb_pred = xgb_2_v4.predict(test_X)

# Bus Bunching 50%-3%
accuracy = accuracy_score(test_Y.busBunching.values, xgb_pred)
precision = precision_score(test_Y.busBunching.values, xgb_pred)
recall = recall_score(test_Y.busBunching.values, xgb_pred)
f_measure = f1_score(test_Y.busBunching.values, xgb_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9333905702225084
Precision: 0.6804384485666105
Recall: 0.5938189845474614
F-measure: 0.6341846758349705


In [20]:
xgb_pred = xgb_2_v3.predict(test_X)

# Bus Bunching 50%-2%
accuracy = accuracy_score(test_Y.busBunching.values, xgb_pred)
precision = precision_score(test_Y.busBunching.values, xgb_pred)
recall = recall_score(test_Y.busBunching.values, xgb_pred)
f_measure = f1_score(test_Y.busBunching.values, xgb_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.943478571939615
Precision: 0.7847847847847848
Recall: 0.5768947755702722
F-measure: 0.6649703138252756


In [14]:
xgb_pred = xgb_2_v2.predict(test_X)

# Bus Bunching 50%-1%
accuracy = accuracy_score(test_Y.busBunching.values, xgb_pred)
precision = precision_score(test_Y.busBunching.values, xgb_pred)
recall = recall_score(test_Y.busBunching.values, xgb_pred)
f_measure = f1_score(test_Y.busBunching.values, xgb_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9483437075194964
Precision: 0.8503850385038504
Recall: 0.5688005886681383
F-measure: 0.681657848324515


In [9]:
xgb_pred = xgb_2_v2.predict(test_X)

# Bus Bunching 50%-50%
accuracy = accuracy_score(test_Y.busBunching.values, xgb_pred)
precision = precision_score(test_Y.busBunching.values, xgb_pred)
recall = recall_score(test_Y.busBunching.values, xgb_pred)
f_measure = f1_score(test_Y.busBunching.values, xgb_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9390074931856335
Precision: 0.8725590592888167
Recall: 0.6405371076029759
F-measure: 0.7387586701745994


In [36]:
xgb_pred = xgb_1.predict(test_X)

# Bus Bunching 50%
accuracy = accuracy_score(test_Y.busBunching.values, xgb_pred)
precision = precision_score(test_Y.busBunching.values, xgb_pred)
recall = recall_score(test_Y.busBunching.values, xgb_pred)
f_measure = f1_score(test_Y.busBunching.values, xgb_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9567861486728196
Precision: 0.9180509413067552
Recall: 0.6100073583517293
F-measure: 0.7329796640141468


#### Random Forest

In [12]:
random.seed(42)
rf_1 = RandomForestClassifier(n_estimators=100, min_samples_split=5,
                           max_features=0.8, n_jobs=-1, warm_start=True)
rf_1.fit(train_X_1, train_Y_1)

/home/veruska_santos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=0.8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=True)

In [21]:
random.seed(42)
rf1_time = time.time()
rf_1.fit(first_1perc_X, first_1perc_Y)
print("Execution time (min): " + str(time.time() - rf1_time))

Execution time (min): 0.004337310791015625


/home/veruska_santos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


In [29]:
random.seed(42)
rf2_time = time.time()
rf_1.fit(second_1perc_X, second_1perc_Y)
print("Execution time (min): " + str(time.time() - rf2_time))

Execution time (min): 0.002845287322998047


/home/veruska_santos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


In [37]:
random.seed(42)
rf3_time = time.time()
rf_1.fit(third_1perc_X, third_1perc_Y)
print("Execution time (min): " + str(time.time() - rf3_time))

Execution time (min): 0.0033724308013916016


/home/veruska_santos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


In [38]:
# Bus Bunching 50%-3% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9345402003999623
Precision: 0.8786246418338108
Recall: 0.5961583326852788
F-measure: 0.7103409933283915


In [30]:
# Bus Bunching 50%-2% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9345402003999623
Precision: 0.8786246418338108
Recall: 0.5961583326852788
F-measure: 0.7103409933283915


In [22]:
# Bus Bunching 50%-1% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9345402003999623
Precision: 0.8786246418338108
Recall: 0.5961583326852788
F-measure: 0.7103409933283915


In [13]:
# Bus Bunching 50% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9345402003999623
Precision: 0.8786246418338108
Recall: 0.5961583326852788
F-measure: 0.7103409933283915


In [30]:
rf_pred = rf_1.predict(test_X)

# Bus Bunching 50%-3%
accuracy = accuracy_score(test_Y.busBunching.values, rf_pred)
precision = precision_score(test_Y.busBunching.values, rf_pred)
recall = recall_score(test_Y.busBunching.values, rf_pred)
f_measure = f1_score(test_Y.busBunching.values, rf_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9559991414466624
Precision: 0.903470715835141
Recall: 0.6129506990434143
F-measure: 0.7303814116615519


In [22]:
rf_pred = rf_1.predict(test_X)

# Bus Bunching 50%-2%
accuracy = accuracy_score(test_Y.busBunching.values, rf_pred)
precision = precision_score(test_Y.busBunching.values, rf_pred)
recall = recall_score(test_Y.busBunching.values, rf_pred)
f_measure = f1_score(test_Y.busBunching.values, rf_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9559991414466624
Precision: 0.903470715835141
Recall: 0.6129506990434143
F-measure: 0.7303814116615519


In [8]:
rf_pred = rf_1.predict(test_X)

# Bus Bunching 50%-1%
accuracy = accuracy_score(test_Y.busBunching.values, rf_pred)
precision = precision_score(test_Y.busBunching.values, rf_pred)
recall = recall_score(test_Y.busBunching.values, rf_pred)
f_measure = f1_score(test_Y.busBunching.values, rf_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9559991414466624
Precision: 0.903470715835141
Recall: 0.6129506990434143
F-measure: 0.7303814116615519


In [12]:
rf_pred = xgb_2_v2.predict(test_X)

# Bus Bunching 50%-50%
accuracy = accuracy_score(test_Y.busBunching.values, rf_pred)
precision = precision_score(test_Y.busBunching.values, rf_pred)
recall = recall_score(test_Y.busBunching.values, rf_pred)
f_measure = f1_score(test_Y.busBunching.values, rf_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9390074931856335
Precision: 0.8725590592888167
Recall: 0.6405371076029759
F-measure: 0.7387586701745994


In [38]:
rf_pred = rf_1.predict(test_X)

# Bus Bunching 50%
accuracy = accuracy_score(test_Y.busBunching.values, rf_pred)
precision = precision_score(test_Y.busBunching.values, rf_pred)
recall = recall_score(test_Y.busBunching.values, rf_pred)
f_measure = f1_score(test_Y.busBunching.values, rf_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.957072333118695
Precision: 0.9129488574537541
Recall: 0.6173657100809419
F-measure: 0.736611062335382


In [39]:
final_predictions = []

for i in range(0, len(rf_pred)):
    temp_preds = []
    temp_preds.append(rf_pred[i])
    temp_preds.append(cat_pred[i])
    temp_preds.append(xgb_pred[i])
    
    final_predictions.append(
        max(temp_preds,key=temp_preds.count)
    )

In [40]:
# Bus Bunching 50%-3% all data
get_quality(final_predictions)

Accuracy: 0.9190966296369976
Precision: 0.8795857988165681
Recall: 0.46239987557352824
F-measure: 0.6061471022987919


In [32]:
# Bus Bunching 50%-2% all data
get_quality(final_predictions)

Accuracy: 0.9225169006760271
Precision: 0.8739495798319328
Recall: 0.4960468673043523
F-measure: 0.632877482512857


In [24]:
# Bus Bunching 50%-1% all data
get_quality(final_predictions)

Accuracy: 0.9257417276556834
Precision: 0.8769720212673233
Recall: 0.5216320605542163
F-measure: 0.6541618581668646


In [15]:
# Bus Bunching 50% all data
get_quality(final_predictions)

Accuracy: 0.9361037528749472
Precision: 0.8803527866391443
Recall: 0.608056614044638
F-measure: 0.7192971696666768


In [32]:
# Bus Bunching 50%-3%
accuracy = accuracy_score(test_Y.busBunching.values, final_predictions)
precision = precision_score(test_Y.busBunching.values, final_predictions)
recall = recall_score(test_Y.busBunching.values, final_predictions)
f_measure = f1_score(test_Y.busBunching.values, final_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9497746297488732
Precision: 0.841831425598335
Recall: 0.5952906548933039
F-measure: 0.6974137931034482


In [24]:
# Bus Bunching 50%-2%
accuracy = accuracy_score(test_Y.busBunching.values, final_predictions)
precision = precision_score(test_Y.busBunching.values, final_predictions)
recall = recall_score(test_Y.busBunching.values, final_predictions)
f_measure = f1_score(test_Y.busBunching.values, final_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9530657508764399
Precision: 0.8627450980392157
Recall: 0.6151582045621781
F-measure: 0.7182130584192441


In [16]:
# Bus Bunching 50%-1%
accuracy = accuracy_score(test_Y.busBunching.values, final_predictions)
precision = precision_score(test_Y.busBunching.values, final_predictions)
recall = recall_score(test_Y.busBunching.values, final_predictions)
f_measure = f1_score(test_Y.busBunching.values, final_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9522787436502826
Precision: 0.8922902494331065
Recall: 0.579102281089036
F-measure: 0.7023650156180277


In [14]:
# Bus Bunching 50%-50%
accuracy = accuracy_score(test_Y.busBunching.values, final_predictions)
precision = precision_score(test_Y.busBunching.values, final_predictions)
recall = recall_score(test_Y.busBunching.values, final_predictions)
f_measure = f1_score(test_Y.busBunching.values, final_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9390074931856335
Precision: 0.8725590592888167
Recall: 0.6405371076029759
F-measure: 0.7387586701745994


In [40]:
# Bus Bunching 50%
accuracy = accuracy_score(test_Y.busBunching.values, final_predictions)
precision = precision_score(test_Y.busBunching.values, final_predictions)
recall = recall_score(test_Y.busBunching.values, final_predictions)
f_measure = f1_score(test_Y.busBunching.values, final_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9583601631251342
Precision: 0.9155080213903743
Recall: 0.6298749080206034
F-measure: 0.7462946817785527
